In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
# Może chwilę potrwać
spark = SparkSession.builder.appName("DataManipulation").getOrCreate()
spark

# Sprawdzanie duplikatów, brakujących obserwacji, outlierów

### Duplikaty

Przyjrzyjmy się poniższej tabeli:

In [6]:
df = spark.createDataFrame([
(1, 144.5, 5.9, 33, 'M'),
(2, 167.2, 5.4, 45, 'M'),
(3, 124.1, 5.2, 23, 'F'),
(4, 144.5, 5.9, 33, 'M'),
(5, 133.2, 5.7, 54, 'F'),
(3, 124.1, 5.2, 23, 'F'),
(5, 129.2, 5.3, 42, 'M'),
], ['id', 'weight', 'height', 'age', 'gender'])

df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



Jak widać mamy kilka problemów: 
* dwa wiersze o ID = 3, które mają te same wartości również w pozostałych kolumnach
* wiersze o ID = 1 i ID = 4 są prawie takie same, różnią się tylko wartością ID. Dlatego możemy na spokojnie założyć, że to ta sama osoba

Ten zbiór danych jest, oczywiście, bardzo mały, więc widać to na pierwszy rzut oka. Co zrobić jeśli mamy miliony rekordów? Warto zacząć od sprawdzenia ile jest duplikatów w zbiorze, porównując wynik funkcji *count()* na pełnym zbiorze z wynikiem funkcji *distinct()* 

In [7]:
print('Count of rows: {0}'.format(df.count()))
print('Count of distinct rows:{0}'.format(df.distinct().count()))

Count of rows: 7
Count of distinct rows:6


Jeżeli te dwa wyniki są różne - już wiadomo, że mamy duble. Można się ich pozbyć funkcją *.dropDuplicates()*

In [ ]:
df = df.dropDuplicates()

Usunęliśmy w ten sposób duble wiersza o ID = 3. Pora na analizę pozostałych kolumn. Można to zrobić operując na okrojonej ramce - bez atrybutu ID.

In [ ]:
print('Count of ids: {0}'.format(df.count()))
print('Count of distinct ids: {0}'.format(
df.select([
c for c in df.columns if c != 'id'
]).distinct().count())
)

Count of ids: 6
Count of distinct ids: 5


Podobnie jak poprzednio, możemy używć funkcji *.dropDuplicates()*, ale wykluczając z operacji ID:

In [ ]:
df = df.dropDuplicates(subset=[
c for c in df.columns if c != 'id'
])

Paramter *subset* sprawia, że *.dropDuplicates()* szuka duplikatów używając jedynie kolumn wymienionych w parametrze. W przykładzie powyżej pozbywamy się wierszy o tych samych wartościach wagi, wzrostu, wieku i płci, ale nie id. Spórzjmy na poniższy wynik:

In [ ]:
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



Uzyskaliśmy oczyszczony zbiór danych - nie ma duplikatów wierszu o ID = 3, a także nie ma wiersza 1/4. 

Skoro już wiemy, że nie zdublowanych (całych) wierszy ani nie ma wierszy łudząco do siebie podobnych można sprawdzić liczbę unikalnych IDków. Zrobimy to wykorzystując funkcję .agg(...):

In [ ]:
import pyspark.sql.functions as fn
df.agg(
fn.count('id').alias('count'),
fn.countDistinct('id').alias('distinct')
).show()

+-----+--------+
|count|distinct|
+-----+--------+
|    5|       4|
+-----+--------+



Powyższy przykład rozpoczęliśmy od zaimportowania wszystkich potrzebnych funkcji z *pyspark.sql module*

Następnie użyliśmy funkcji *.count()* i *.countDistinct()* żeby sprawdzić liczbę wierszy i liczbę unikalnych IDs. Funckja *.alias()* umożliwiła nadanie wynikowi innej nazwy.

Jak widać powyżej mamy 5 wierszy, ale tylko 4 unikalne ID. W poprzednich krokach pozbyliśmy się duplikatów, więc możemy założyć, że nastąpiło jakieś zamieszanie w danych i żeby to wyprostować nadamy wierszom nowe ID:

In [ ]:
df.withColumn('new_id',
fn.monotonically_increasing_id()).show()

+---+------+------+---+------+------+
| id|weight|height|age|gender|new_id|
+---+------+------+---+------+------+
|  5| 133.2|   5.7| 54|     F|     0|
|  1| 144.5|   5.9| 33|     M|     1|
|  2| 167.2|   5.4| 45|     M|     2|
|  3| 124.1|   5.2| 23|     F|     3|
|  5| 129.2|   5.3| 42|     M|     4|
+---+------+------+---+------+------+



Funkcją *.monotonicallymonotonically_increasing_id()* nadaliśmy nowe, rosnące ID. 

### Brakujące obserwacje

Żeby nie zostać ze zbyt małą liczbą obserwacji w zbiorze danych warto sprawdzić, które atrybuty mają największe ubytki i rozważyć wykluczenie ich z analizy. Poza tym - jeśli zmienna ma prawie same braki jest bezużyteczna modelarsko. 

Inną metodą na poradzenie sobie z brakami jest imputacja. Można to zrobić na kilka sposobów: 
* dla danych Boolean - dodać kolejną kategorię np. "missing" 
* jeżeli dane już są kategoryczne - dodać jeszcze jeden poziom, "missing" właśnie
* w przypadku danych porządkorwych albo numerycznych można imputować średnią, medianę, kwartylem, percentylem etc. 

Rozważmy przykład:

In [ ]:
df_miss = spark.createDataFrame([ (1, 143.5, 5.6, 28,
'M', 100000),
(2, 167.2, 5.4, 45, 'M', None),
(3, None , 5.2, None, None, None),
(4, 144.5, 5.9, 33, 'M', None),
(5, 133.2, 5.7, 54, 'F', None),
(6, 124.1, 5.2, None, 'F', None),
(7, 129.2, 5.3, 42, 'M', 76000),
], ['id', 'weight', 'height', 'age', 'gender', 'income'])

df_miss.show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  1| 143.5|   5.6|  28|     M|100000|
|  2| 167.2|   5.4|  45|     M|  null|
|  3|  null|   5.2|null|  null|  null|
|  4| 144.5|   5.9|  33|     M|  null|
|  5| 133.2|   5.7|  54|     F|  null|
|  6| 124.1|   5.2|null|     F|  null|
|  7| 129.2|   5.3|  42|     M| 76000|
+---+------+------+----+------+------+



W przykładzie powyżej mamy do czynienia z kilkoma kategoriami missingów. Analizując wiersze widać, że: 

* wiersz z ID = 3 ma tylko jedną użyteczną informację - wzrost
* kolumna "income" (dochód), w związku z tym, że to delikatna kwestia, ma bardzo dużo braków
* wzrost i płeć mają tylko po jednym braku
* wzrost ma dwa missingi

Żeby policzyć missingi per wiersz można skorzystać z kodu poniżej: 

In [ ]:
df_miss.rdd.map(
lambda row: (row['id'], sum([c == None for c in row]))
).collect()

[(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]

Wynik należy czytać tak: wiersz o ID = 3 ma 4 brakujące wartości. Sprawdźmy co to za wartości żeby móc zdecydować czy je droppujemy czy coś imputujemy:

In [ ]:
df_miss.where('id == 3').show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  3|  null|   5.2|null|  null|  null|
+---+------+------+----+------+------+



Sprawdźmy teraz udział % missingów w każdej kolumnie:

In [ ]:
df_miss.agg(*[
(1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing')
for c in df_miss.columns
]).show()

+----------+------------------+--------------+------------------+------------------+------------------+
|id_missing|    weight_missing|height_missing|       age_missing|    gender_missing|    income_missing|
+----------+------------------+--------------+------------------+------------------+------------------+
|       0.0|0.1428571428571429|           0.0|0.2857142857142857|0.1428571428571429|0.7142857142857143|
+----------+------------------+--------------+------------------+------------------+------------------+



Mamy 14% missingów we wzroście i aż 72% missingów w przypadku dochodu. To dość jasne co należy z tym zrobić. Najpierw zdroppujemy dochód. 

In [ ]:
df_miss_no_income = df_miss.select([
c for c in df_miss.columns if c != 'income'
])

Dzięki temu już wiadomo, że nie trzeba usuwać wiersza ID = 3, bo kolumny "wzrost" i "wiek" mają wystarczająco dużo obserwacji żeby wyliczyć średnią i zastąpić nią braki. Gdybyśmy zdecydowali się je usunąć powinniśmy skorzystać z fukcji *.dropna(...)*, tak jak w przykładzie poniżej. W nim użyliśmy też parametru *tresh*, który pomaga wyspecyfikować jaki jest nasz poziom tolerancji na braków per wiersz. Parametr *tresh* przydaje się zwłaszcza wtedy kiedy mamy miliony rekordów i setki zmiennych:

In [ ]:
df_miss_no_income.dropna(thresh=3).show()

Z drugiej strony, gdyby zależało nam na imputacji brakujących obserwacji, moglibyśmy użyć funkcji *.fillna()*. Funkcja przyjmuje integery, floaty albo stringi. Wszystkie braki zostaną wypełnione przekazanym parametrem. Można jej także przekazać słownik: {'<kolumna>':'<wartość do zaimputowania>'}. Na słowniki nałożone są te same ograniczenia, co na *.fillna()*. 
Jeżeli chcemy zaimputować braki średnią czy medianą, musimy tę wartość najpierw policzyć albo stworzyć słownik wartości i potem przekazać je do *.fillna()*

Poniżej przykład jak to zrobić:

In [ ]:
means = df_miss_no_income.agg(
*[fn.mean(c).alias(c)
for c in df_miss_no_income.columns if c != 'gender']
).toPandas().to_dict('records')[0]
means['gender'] = 'missing'
df_miss_no_income.fillna(means).show()

+---+------------------+------+---+-------+
| id|            weight|height|age| gender|
+---+------------------+------+---+-------+
|  1|             143.5|   5.6| 28|      M|
|  2|             167.2|   5.4| 45|      M|
|  3|140.28333333333333|   5.2| 40|missing|
|  4|             144.5|   5.9| 33|      M|
|  5|             133.2|   5.7| 54|      F|
|  6|             124.1|   5.2| 40|      F|
|  7|             129.2|   5.3| 42|      M|
+---+------------------+------+---+-------+



Pominęliśmy kolumnę "płeć" z wiadomych powodów. 
W przykładzie powyżej zastosowaliśmy podwójną konswersję - najpierw wynik .agg(...) (ramka PySparkowa) przekonwertowaliśmy  na ramkę pandasową, a potem na słownik. 

Parametr .to_dict(), pochodzący z pandas, instruuje sparka żeby stworzył powyższy słownik.

### Outliers

Outliery to obserwacje, które nie mieszą się w przedziale <Q1−1.5IQR; Q3+1.5IQR>. Spójrzmy na poniższy przykład:

In [ ]:
df_outliers = spark.createDataFrame([
(1, 143.5, 5.3, 28),
(2, 154.2, 5.5, 45),
(3, 342.3, 5.1, 99),
(4, 144.5, 5.5, 33),
(5, 133.2, 5.4, 54),
(6, 124.1, 5.1, 21),
(7, 129.2, 5.3, 42),
], ['id', 'weight', 'height', 'age'])

df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



Teraz możemy wykorzystać rozstęp żeby oflagować outliery. Użyjemy do tego *.approxQuantile(...)*. Pierwszy argument, który przekażemy to nazwa kolumny, kolejnym może być albo numer między 0 a 1 (gdzie 0.5 oznacza policz medianę) albo lista (czyli to, co my zrobimy). Trzeci parametr oznacza poziom błędu, na który jesteśmy w stanie się zgodzić. Ustawienie go sprawi, że otrzymamy bardzo dokładny wynik, ale może być bardzo ciężki obliczeniowo.

In [53]:
cols = ['weight', 'height', 'age'] ### nie mogę iteracyjnie dodać kolejnej wartości w słowniku
bounds = {}
for col in cols:
    quantiles = df_outliers.approxQuantile(
    col, [0.25, 0.75], 0.05
)
IQR = quantiles[1] - quantiles[0]
curr_bound = {col:(quantiles[0] - 1.5 * IQR,
quantiles[1] + 1.5 * IQR)}
bounds.update(curr_bound)

The bounds dictionary holds the lower and upper bounds for each
feature:

In [54]:
bounds

{'age': (-11.0, 93.0)}

Użyjmy go do oznaczenia outlierów:

In [34]:
outliers = df_outliers.select(*['id'] + [
(
(df_outliers[c] < bounds[c][0]) |
(df_outliers[c] > bounds[c][1])
).alias(c + '_o') for c in cols])
outliers.show()

KeyError: 'weight'

Mamy dwa outliery w wadze i dwa w wieku. Poniżej kod, który zwróci wartości istotnie różne od pozostałych:

In [ ]:
df_outliers = df_outliers.join(outliers, on='id')
df_outliers.filter('weight_o').select('id', 'weight').show()
df_outliers.filter('age_o').select('id', 'age').show()

Powyższy kod zwróci poniższą ramkę:

Mając informacje przytoczone w tej sekcji jesteśmy w szybko wyczyścić nawet bardzo duży zbiór danych.

# Wstępne rozeznanie co siedzi w danych 

## Descriptive statistics
Warto zacząć od statystyki opisowej, którą sprawdziwmy podstawowe informacje o zbiorze: średnie, odchylenia standardowe, wartości min i max itd. 
Zaczniemy od załadowania danych i przekonwertowania ich na ramkę Sparkową:


First, we load the only module we will need. The pyspark.sql.types
exposes all the data types we can use, such as IntegerType() or
FloatType().

Next, we read the data in and remove the header line using the
.filter(...) method. This is followed by splitting the row on each
comma (since this is a .csv file) and converting each element to an
integer:

In [4]:
fraud = spark.sparkContext.textFile('ccFraud.csv.gz')
header = fraud.first()
fraud = fraud \
.filter(lambda row: row != header) \
.map(lambda row: [int(elem) for elem in row.split(',')])

Next, we create the schema for our DataFrame:

In [ ]:
fields = [
*[
typ.StructField(h[1:-1], typ.IntegerType(), True)
for h in header.split(',')
]
] schema = typ.StructType(fields)

Finally, we create our DataFrame:

In [ ]:
fraud_df = spark.createDataFrame(fraud, schema)

Having created our fraud_df DataFrame, we can calculate the basic 
descriptive statistics for our dataset. However, you need to remember
that even though all of our features appear as numeric in nature, some of
them are categorical (for example, gender or state).
Here's the schema of our DataFrame:

In [ ]:
fraud_df.printSchema()

Also, no information would be gained from calculating the mean and
standard deviation of the custId column, so we will not be doing that.
For a better understanding of categorical columns, we will count the
frequencies of their values using the .groupby(...) method. In this
example, we will count the frequencies of the gender column:

In [ ]:
fraud_df.groupby('gender').count().show()

The preceding code will produce the following output:

As you can see, we are dealing with a fairly imbalanced dataset. What
you would expect to see is an equal distribution for both genders.
Note
It goes beyond the scope of this chapter, but if we were building a
statistical model, you would need to take care of these kinds of biases.
For the truly numerical features, we can use the .describe() method:

In [ ]:
numerical = ['balance', 'numTrans', 'numIntlTrans']
desc = fraud_df.describe(numerical)
desc.show()

Even from these relatively few numbers we can tell quite a bit:
All of the features are positively skewed. The maximum values are a
number of times larger than the average.
The coefficient of variation (the ratio of mean to standard deviation)
is very high (close or greater than 1), suggesting a wide spread of
observations.
Here's how you check the skeweness (we will do it for the 'balance'
feature only):

In [ ]:
fraud_df.agg({'balance': 'skewness'}).show()

A list of aggregation functions (the names are fairly self-explanatory)
includes: avg(), count(), countDistinct(), first(), kurtosis(),
max(), mean(), min(), skewness(), stddev(), stddev_pop(),
stddev_samp(), sum(), sumDistinct(), var_pop(), var_samp() and
variance().

### Correlations
Another highly useful measure of mutual relationships between features
is correlation. Your model would normally include only those features
that are highly correlated with your target. However, it is almost equally
important to check the correlation between the features; including
features that are highly correlated among them (that is, are collinear)
may lead to unpredictable behavior of your model, or might unnecessarily complicate it.

Calculating correlations in PySpark is very easy once your data is in a
DataFrame form. The only difficulties are that the .corr(...) method
supports the Pearson correlation coefficient at the moment, and it can
only calculate pairwise correlations, such as the following:

In [ ]:
fraud_df.corr('balance', 'numTrans')

In order to create a correlations matrix, you can use the following script:

In [ ]:
n_numerical = len(numerical)
corr = []
for i in range(0, n_numerical):
temp = [None] * i
for j in range(i, n_numerical):
temp.append(fraud_df.corr(numerical[i], numerical[j]))
corr.append(temp)

As you can see, the correlations between the numerical features in the
credit card fraud dataset are pretty much non-existent. Thus, all these
features can be used in our models, should they turn out to be statistically sound in explaining our target.
Having checked the correlations, we can now move on to visually
inspecting our data.

# Visualization
There are multiple visualization packages, but in this section we will be
using matplotlib and Bokeh exclusively to give you the best tools for
your needs.
Both of the packages come preinstalled with Anaconda. First, let's load
the modules and set them up:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import bokeh.charts as chrt
from bokeh.io import output_notebook
output_notebook()

The %matplotlib inline and the output_notebook() commands will
make every chart generated with matplotlib or Bokeh, respectively,
appear within the notebook and not as a separate window.
Histograms
Histograms are by far the easiest way to visually gauge the distribution
of your features. There are three ways you can generate histograms in
PySpark (or a Jupyter notebook):

Aggregate the data in workers and return an aggregated list of bins
and counts in each bin of the histogram to the driver
Return all the data points to the driver and allow the plotting
libraries' methods to do the job for you
Sample your data and then return them to the driver for plotting.
If the number of rows in your dataset is counted in billions, then the
second option might not be attainable. Thus, you need to aggregate the
data first:

In [ ]:
hists = fraud_df.select('balance').rdd.flatMap(lambda row: row
).histogram(20)

To plot the histogram, you can simply call matplotlib, as shown in the
following code:

In [ ]:
data = {
'bins': hists[0][:-1],
'freq': hists[1]
} plt.bar(data['bins'], data['freq'], width=2000)
plt.title('Histogram of \'balance\'')

In a similar manner, a histogram can be created with Bokeh:

In [ ]:
b_hist = chrt.Bar(data,
values='freq', label='bins',
title='Histogram of \'balance\'')
chrt.show(b_hist)

Since Bokeh uses D3.js in the background, the resulting chart is
interactive:

If your data is small enough to fit on the driver (although we would
argue it would normally be faster to use the previous method), you can
bring the data and use the .hist(...) (from matplotlib) or
.Histogram(...) (from Bokeh) methods:

In [ ]:
data_driver = {
'obs': fraud_df.select('balance').rdd.flatMap(
lambda row: row
).collect()
}
plt.hist(data_driver['obs'], bins=20)
plt.title('Histogram of \'balance\' using .hist()')
b_hist_driver = chrt.Histogram(
data_driver, values='obs',
title='Histogram of \'balance\' using .Histogram()',
bins=20
) chrt.show(b_hist_driver)

# Interactions between features
Scatter charts allow us to visualize interactions between up to three
variables at a time (although we will be only presenting a 2D interaction
in this section).

Since PySpark does not offer any visualization modules on the server
side, and trying to plot billions of observations at the same time would be highly impractical, in this section we will sample the dataset at 0.02%
(roughly 2,000 observations).

In this example, we will sample our fraud dataset at 0.02% given
'gender' as a strata:

In [ ]:
data_sample = fraud_df.sampleBy(
'gender', {1: 0.0002, 2: 0.0002}
).select(numerical)

To put multiple 2D charts in one go, you can use the following code:

In [ ]:
data_multi = dict([
(elem, data_sample.select(elem).rdd \
.flatMap(lambda row: row).collect())
for elem in numerical
])
sctr = chrt.Scatter(data_multi, x='balance', y='numTrans')
chrt.show(sctr)

The preceding code will produce the following chart:

As you can see, there are plenty of fraudulent transactions that had 0
balance but many transactions—that is, a fresh card and big spike of
transactions. However, no specific pattern can be shown apart from
some banding occurring at $1,000 intervals.

# Summary
In this chapter, we looked at how to clean and prepare your dataset for
modeling by identifying and tackling datasets with missing values,
duplicates, and outliers. We also looked at how to get a bit more familiar
with your data using tools from PySpark (although this is by no means a
full manual on how to analyze your datasets). Finally, we showed you
how to chart your data.
We will use these (and more) techniques in the next two chapters, where
we will be building machine learning models.